<a href="https://colab.research.google.com/github/mlabonne/llm-course/blob/main/Fine_tune_Llama_2_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 2 in Google Colab
> 🗣️ Large Language Model Course

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne), based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da). Special thanks to Tolga HOŞGÖR for his solution to empty the VRAM.

This notebook runs on a T4 GPU. (Last update: 01 Aug 2023)


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
'''
这五个库在 Python 中的作用如下：

1. `accelerate`: 这是一个由 Hugging Face 提供的库，用于简化在多种硬件上进行深度学习训练的过程。它可以帮助你轻松地在 CPU、单个 GPU、多个 GPU 或 TPU 上运行你的 PyTorch 代码。

2. `peft`: 我找不到名为 "peft" 的 Python 库，可能是一个错误或者是一个私有库。你可能需要检查一下这个库的名称。

3. `bitsandbytes`: 这是一个用于优化深度学习训练的库。它提供了一种新的优化器状态管理方法，可以显著减少 GPU 内存使用量，并提高训练速度。

4. `transformers`: 这是一个由 Hugging Face 提供的库，用于处理预训练的 transformer 模型。它提供了大量的预训练模型，如 BERT、GPT-2、RoBERTa 等，以及对应的模型架构和分词器。

5. `trl`: 这是一个用于强化学习的库，提供了一些用于训练强化学习模型的工具。'''

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
'''
这段代码是从 `transformers` 库中导入了一些类和函数。`transformers` 是一个由 Hugging Face 开发的库，它提供了大量预训练的模型和相关工具，用于处理各种自然语言处理（NLP）任务。

- `AutoModelForCausalLM` 是一个用于因果语言建模的类。这种模型可以用于生成文本，例如聊天机器人或写作助手。

- `AutoTokenizer` 是一个用于自动化处理和编码文本的类。它可以自动选择和加载正确的词汇表和编码规则，以匹配预训练模型的需求。

- `BitsAndBytesConfig` 是一个配置类，用于处理和管理模型的配置设置。

- `HfArgumentParser` 是一个用于解析命令行参数的类。它可以将命令行参数转换为特定的数据类型。

- `TrainingArguments` 是一个用于管理训练过程中的各种参数的类。

- `pipeline` 是一个函数，用于创建处理模型输入和输出的管道。这些管道可以用于各种任务，如文本分类、命名实体识别、问答等。

- `logging` 是一个模块，用于记录和管理日志信息。这对于跟踪和调试模型训练过程非常有用。
'''

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"
'''
这行代码定义了一个变量 `model_name`，并将其设置为字符串 "NousResearch/Llama-2-7b-chat-hf"。在 Hugging Face 的 `transformers` 库中，这种字符串通常用于指定预训练模型的名称。

在这个例子中，"NousResearch/Llama-2-7b-chat-hf" 是一个预训练模型的标识符，它由两部分组成：模型的创建者（"NousResearch"）和模型的具体名称（"Llama-2-7b-chat-hf"）。这个模型存储在 Hugging Face 的模型库中，可以通过这个标识符直接加载。

这个模型可能是为聊天或对话系统预训练的，因为名称中包含了 "chat"。然而，具体的模型类型和用途，以及它是否适合你的任务，需要查看模型在 Hugging Face 模型库中的具体文档和描述。
'''
# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"
'''
这行代码定义了一个变量 `dataset_name`，并将其设置为字符串 "mlabonne/guanaco-llama2-1k"。在 Hugging Face 的 `datasets` 库中，这种字符串通常用于指定数据集的名称。

在这个例子中，"mlabonne/guanaco-llama2-1k" 是一个数据集的标识符，它由两部分组成：数据集的创建者（"mlabonne"）和数据集的具体名称（"guanaco-llama2-1k"）。这个数据集存储在 Hugging Face 的数据集库中，可以通过这个标识符直接加载。

这个数据集可能包含了一千个（由 "1k" 这个部分暗示）样本，用于训练或微调 "Llama-2" 模型。然而，具体的数据集内容和用途，以及它是否适合你的任务，需要查看数据集在 Hugging Face 数据集库中的具体文档和描述。
'''
# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"
'''
这行代码定义了一个变量 `new_model`，并将其设置为字符串 "llama-2-7b-miniguanaco"。在这个上下文中，这个字符串可能代表一个新的模型名称。

在训练或微调模型后，通常需要将模型保存以便于后续使用或分享。这个字符串 "llama-2-7b-miniguanaco" 可能就是将要保存的模型的名称。这个名称可能反映了模型的一些特性，例如它可能是基于 "Llama-2-7b" 模型微调的，而 "miniguanaco" 可能表示这个模型是在 "guanaco" 数据集的一个子集上进行微调的。

然而，具体的含义和用途可能会根据代码的其他部分和你的项目需求有所不同。这只是根据这一行代码的内容和常见的命名习惯进行的推测。
'''
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64
'''
这段代码定义了一个变量 `lora_r`，并将其设置为整数 64。这个变量可能用于设置 QLoRA（Quantized Latent Representation Attention）模型的参数。
QLoRA 是一种自然语言处理（NLP）模型，它使用了一种名为 LoRA（Latent Representation Attention）的注意力机制。在这个机制中，"attention dimension" 是一个重要的参数，它决定了注意力向量的维度。
在这个例子中，`lora_r` 被设置为 64，这意味着在 QLoRA 模型中，注意力向量的维度将被设置为 64。这个参数的设置可能会影响模型的性能和计算需求。较大的维度可能会提高模型的性能，但也会增加计算的复杂性和内存需求。因此，选择合适的维度是一个需要权衡的问题。
'''
# Alpha parameter for LoRA scaling
lora_alpha = 16
'''
这段代码定义了一个变量 `lora_alpha`，并将其设置为整数 16。这个变量可能用于设置 LoRA（Latent Representation Attention）模型的参数。

LoRA 是一种自然语言处理（NLP）模型，它使用了一种名为 "Latent Representation Attention" 的注意力机制。在这个机制中，"alpha" 是一个重要的参数，它用于控制 LoRA 的缩放因子。

在这个例子中，`lora_alpha` 被设置为 16，这意味着在 LoRA 模型中，缩放因子将被设置为 16。这个参数的设置可能会影响模型的性能。"alpha" 参数的具体作用可能会根据 LoRA 模型的具体实现和使用方式有所不同，但通常，它会影响模型中注意力的分布和缩放。
'''
# Dropout probability for LoRA layers
lora_dropout = 0.1
'''
这段代码定义了一个变量 `lora_dropout`，并将其设置为 0.1。这个变量可能用于设置 LoRA（Latent Representation Attention）模型中的 dropout 概率。

Dropout 是一种常用的正则化技术，它在训练过程中随机丢弃（即设置为零）神经网络层的一部分输出。这可以帮助防止模型过拟合，即模型在训练数据上表现良好，但在未见过的测试数据上表现较差。
在这个例子中，`lora_dropout` 被设置为 0.1，这意味着在 LoRA 模型的每个层中，大约有 10% 的输出在每次训练步骤中会被随机设置为零。这个参数的设置可能会影响模型的性能和泛化能力。较高的 dropout 概率可能会增强模型的泛化能力，但也可能导致模型在训练数据上的性能下降。因此，选择合适的 dropout 概率是一个需要权衡的问题。
'''
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True
'''
这段代码定义了一个变量 `use_4bit`，并将其设置为 `True`。这个变量可能用于决定是否使用 4 位精度进行计算。

在神经网络的训练和推理过程中，我们通常需要进行大量的浮点数运算。这些运算的精度会影响到模型的性能和计算需求。使用更高的精度（如 32 位或 64 位）可以提高模型的性能，但也会增加计算的复杂性和内存需求。相反，使用更低的精度（如 4 位或 8 位）可以减少计算的复杂性和内存需求，但可能会降低模型的性能。

在这个例子中，`use_4bit` 被设置为 `True`，这意味着将使用 4 位精度进行计算。这可能是为了在保持一定模型性能的同时，减少计算的复杂性和内存需求。
'''
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
'''
这行代码定义了一个变量 `bnb_4bit_compute_dtype`，并将其值设置为字符串 `"float16"`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`bnb_4bit_compute_dtype` 可能是用来指定某种计算的数据类型。`"float16"` 是一种浮点数数据类型，它使用 16 位来存储一个数值，包括符号位、指数位和尾数位。这种数据类型可以表示的数值范围较小，精度也较低，但是它的存储空间和计算速度优势使得它在某些特定的应用场景（如深度学习和图形处理）中非常有用。

请注意，这里 `"float16"` 是一个字符串，而不是直接的数据类型。这意味着它可能被用在一个需要动态设置数据类型的上下文中，例如，它可能被传递给一个函数或方法，该函数或方法会根据这个字符串来设置其内部的数据类型。
'''
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
'''
这行代码定义了一个变量 `bnb_4bit_quant_type`，并将其值设置为字符串 `"nf4"`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`bnb_4bit_quant_type` 可能是用来指定某种量化类型。`"nf4"` 是一个字符串，可能代表一种特定的量化策略或方法。量化通常用于深度学习模型，以减少模型的存储和计算需求，同时尽可能地保持模型的性能。

请注意，这里 `"nf4"` 是一个字符串，而不是直接的量化类型。这意味着它可能被用在一个需要动态设置量化类型的上下文中，例如，它可能被传递给一个函数或方法，该函数或方法会根据这个字符串来设置其内部的量化类型。

然而，`"nf4"` 是一个不常见的量化类型标识，可能是特定库或框架中的自定义类型。为了更准确地理解这个值的含义，你可能需要查阅相关的文档或源代码。
'''
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
'''
这行代码定义了一个变量 `use_nested_quant`，并将其值设置为布尔值 `False`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`use_nested_quant` 可能是用来控制是否启用嵌套量化（nested quantization）的开关。嵌套量化是一种特殊的量化策略，通常用于深度学习模型，以进一步减少模型的存储和计算需求。特别地，这里的注释提到了 "4-bit base models" 和 "double quantization"，这可能意味着这种嵌套量化策略是针对 4 位基础模型的双重量化。

设置 `use_nested_quant = False` 表示当前不启用这种嵌套量化策略。如果你想启用这种策略，你可以将 `use_nested_quant` 的值设置为 `True`。

请注意，这里的 `False` 是一个布尔值，而不是字符串。在 Python 中，布尔值只有两种：`True` 和 `False`，分别代表逻辑真和逻辑假。
'''
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"
'''
这行代码定义了一个变量 `output_dir`，并将其值设置为字符串 `"./results"`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`output_dir` 可能是用来指定模型预测结果和检查点（checkpoints）存储的输出目录。字符串 `"./results"` 是一个相对路径，它指向当前工作目录下的一个名为 "results" 的子目录。

检查点是深度学习训练过程中的一个重要概念，它们是在训练过程中的某个时刻保存的模型状态，可以用于后续的模型恢复、微调或预测。将检查点和预测结果保存到磁盘是一个常见的做法，因为这样可以在训练完成后进行进一步的分析和评估。

请注意，如果 `"./results"` 目录在运行代码时不存在，可能需要先创建这个目录，否则在尝试写入文件时可能会出现错误。
'''
# Number of training epochs
num_train_epochs = 1
'''
这行代码定义了一个变量 `num_train_epochs`，并将其值设置为整数 `1`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`num_train_epochs` 是用来指定训练模型的迭代次数。在深度学习中，一个 "epoch" 是指整个训练集通过模型一次的过程。例如，如果你有 1000 个训练样本，那么一个 epoch 就是这 1000 个样本全部都被模型处理一次。

设置 `num_train_epochs = 1` 表示模型将只对整个训练集进行一次训练。这可能是为了快速验证模型的性能，或者是因为训练集非常大，一次迭代就足够训练模型。然而，在实际的深度学习任务中，通常需要多个 epochs 的训练才能使模型收敛到一个好的性能。

请注意，这里的 `1` 是一个整数，而不是字符串。在 Python 中，整数可以直接用于数学运算，而不需要进行任何转换。
'''
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False
'''
这两行代码定义了两个变量 `fp16` 和 `bf16`，并都将它们的值设置为布尔值 `False`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`fp16` 和 `bf16` 可能是用来控制是否启用特定类型的深度学习训练的开关。`fp16` 和 `bf16` 分别代表 16 位浮点数（float16）和 16 位脑浮点数（bfloat16）训练。

设置 `fp16 = False` 和 `bf16 = False` 表示当前不启用 16 位浮点数和 16 位脑浮点数训练。这两种训练方式都可以减少模型的存储和计算需求，同时尽可能地保持模型的性能。特别地，注释中提到了 "set bf16 to True with an A100"，这可能意味着如果你在 NVIDIA A100 GPU 上训练模型，你可以启用 16 位脑浮点数训练。

请注意，这里的 `False` 是一个布尔值，而不是字符串。在 Python 中，布尔值只有两种：`True` 和 `False`，分别代表逻辑真和逻辑假。
'''
# Batch size per GPU for training
per_device_train_batch_size = 4
'''
这行代码定义了一个变量 `per_device_train_batch_size`，并将其值设置为整数 `4`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`per_device_train_batch_size` 是用来指定每个 GPU 用于训练的批量大小。在深度学习中，批量大小（batch size）是指每次模型训练时处理的样本数量。例如，如果你有 1000 个训练样本，批量大小为 4，那么模型将会在每个 epoch 中分 250 次处理这些样本。

设置 `per_device_train_batch_size = 4` 表示每个 GPU 在每次训练时将处理 4 个样本。批量大小的选择会影响模型的训练速度和性能。较大的批量大小可以提高硬件的利用率和训练速度，但可能需要更多的内存，并且可能不会得到最优的模型性能。

请注意，这里的 `4` 是一个整数，而不是字符串。在 Python 中，整数可以直接用于数学运算，而不需要进行任何转换。
'''
# Batch size per GPU for evaluation
per_device_eval_batch_size = 4
'''
这行代码定义了一个变量 `per_device_eval_batch_size`，并将其值设置为整数 `4`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`per_device_eval_batch_size` 是用来指定每个 GPU 用于评估的批量大小。在深度学习中，批量大小（batch size）是指每次模型评估时处理的样本数量。例如，如果你有 1000 个评估样本，批量大小为 4，那么模型将会分 250 次处理这些样本进行评估。

设置 `per_device_eval_batch_size = 4` 表示每个 GPU 在每次评估时将处理 4 个样本。批量大小的选择会影响模型的评估速度和内存使用。较大的批量大小可以提高硬件的利用率和评估速度，但可能需要更多的内存。

请注意，这里的 `4` 是一个整数，而不是字符串。在 Python 中，整数可以直接用于数学运算，而不需要进行任何转换。
'''
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
'''
这行代码定义了一个变量 `gradient_accumulation_steps`，并将其值设置为整数 `1`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`gradient_accumulation_steps` 是用来指定梯度累积的步数。在深度学习中，梯度累积是一种常用的策略，用于在内存限制下处理大批量的数据。具体来说，它将多个小批量的梯度累积起来，然后一次性更新模型参数，这样可以模拟大批量训练的效果。

设置 `gradient_accumulation_steps = 1` 表示每次更新模型参数时，只累积一步的梯度。这实际上就是普通的小批量训练，每次处理一批数据后就立即更新模型参数。如果你想模拟大批量训练，可以将 `gradient_accumulation_steps` 的值设置为大于 1 的整数。

请注意，这里的 `1` 是一个整数，而不是字符串。在 Python 中，整数可以直接用于数学运算，而不需要进行任何转换。
'''
# Enable gradient checkpointing
gradient_checkpointing = True
'''
这行代码定义了一个变量 `gradient_checkpointing`，并将其值设置为布尔值 `True`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`gradient_checkpointing` 可能是用来控制是否启用梯度检查点（gradient checkpointing）的开关。梯度检查点是一种内存优化策略，它在深度学习模型训练中用于减少 GPU 内存的使用。具体来说，它通过在前向传播过程中保存一些中间计算结果，然后在反向传播过程中重新计算其他结果，从而减少了需要同时存储的中间结果的数量。

设置 `gradient_checkpointing = True` 表示启用梯度检查点。这可以在内存有限的情况下训练更大的模型或使用更大的批量大小，但可能会增加一些计算开销，因为需要重新计算一些中间结果。

请注意，这里的 `True` 是一个布尔值，而不是字符串。在 Python 中，布尔值只有两种：`True` 和 `False`，分别代表逻辑真和逻辑假。
'''
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
'''
这行代码定义了一个变量 `max_grad_norm`，并将其值设置为浮点数 `0.3`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`max_grad_norm` 可能是用来指定梯度裁剪（gradient clipping）的阈值。在深度学习中，梯度裁剪是一种常用的策略，用于防止梯度爆炸问题。具体来说，如果一个梯度的范数超过了这个阈值，那么它会被缩放到这个阈值，这样可以保证梯度的范数不会过大。

设置 `max_grad_norm = 0.3` 表示梯度的最大范数为 0.3。如果一个梯度的范数超过了 0.3，那么它会被缩放到 0.3。这个值的选择需要根据具体的任务和模型来调整，一般来说，如果梯度经常被裁剪，那么可能需要增大这个值；反之，如果梯度很少被裁剪，那么可能需要减小这个值。

请注意，这里的 `0.3` 是一个浮点数，而不是字符串。在 Python 中，浮点数可以直接用于数学运算，而不需要进行任何转换。
'''
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
'''
这行代码定义了一个变量 `learning_rate`，并将其值设置为浮点数 `2e-4`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`learning_rate` 是用来指定优化器的初始学习率。在深度学习中，学习率是一个重要的超参数，它控制了模型参数在训练过程中的更新速度。具体来说，学习率决定了每次参数更新时，应该移动多远以减小损失函数。

设置 `learning_rate = 2e-4` 表示优化器的初始学习率为 0.0002。这个值的选择需要根据具体的任务和模型来调整，一般来说，如果模型训练过程中的损失下降得太慢，那么可能需要增大学习率；反之，如果模型训练过程中的损失震荡或上升，那么可能需要减小学习率。

特别地，注释中提到了 "AdamW optimizer"，这可能意味着这个学习率是用于 AdamW 优化器的。AdamW 是一种常用的深度学习优化器，它在 Adam 的基础上加入了权重衰减，可以更好地控制模型的复杂度。

请注意，这里的 `2e-4` 是一个浮点数，而不是字符串。在 Python 中，浮点数可以直接用于数学运算，而不需要进行任何转换。
'''
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
'''
这行代码定义了一个变量 `weight_decay`，并将其值设置为浮点数 `0.001`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`weight_decay` 是用来指定权重衰减（weight decay）的值。在深度学习中，权重衰减是一种常用的正则化策略，用于防止模型过拟合。具体来说，权重衰减通过在损失函数中添加一个与模型权重平方成正比的项，来鼓励模型学习到的权重值更小，从而使模型更简单，更能泛化到未见过的数据。

设置 `weight_decay = 0.001` 表示权重衰减的值为 0.001。这个值的选择需要根据具体的任务和模型来调整，一般来说，如果模型过拟合，那么可能需要增大权重衰减；反之，如果模型欠拟合，那么可能需要减小权重衰减。

特别地，注释中提到了 "to apply to all layers except bias/LayerNorm weights"，这可能意味着这个权重衰减只应用于除偏置和层归一化权重之外的其他权重。这是因为偏置和层归一化权重通常不需要正则化，因为它们对模型复杂度的影响较小。

请注意，这里的 `0.001` 是一个浮点数，而不是字符串。在 Python 中，浮点数可以直接用于数学运算，而不需要进行任何转换。
'''
# Optimizer to use
optim = "paged_adamw_32bit"
'''
这行代码定义了一个变量 `optim`，并将其值设置为字符串 `"paged_adamw_32bit"`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`optim` 可能是用来指定优化器的类型。在深度学习中，优化器是用于更新模型参数以最小化损失函数的算法。常见的优化器包括 SGD（随机梯度下降）、Adam、RMSProp 等。

字符串 `"paged_adamw_32bit"` 可能代表一种特定的优化器。这个优化器可能是 AdamW 的一个变种，并且使用 32 位的浮点数进行计算。AdamW 是一种常用的深度学习优化器，它在 Adam 的基础上加入了权重衰减，可以更好地控制模型的复杂度。

然而，`"paged_adamw_32bit"` 是一个不常见的优化器标识，可能是特定库或框架中的自定义类型。为了更准确地理解这个值的含义，你可能需要查阅相关的文档或源代码。

请注意，这里 `"paged_adamw_32bit"` 是一个字符串，而不是直接的优化器类型。这意味着它可能被用在一个需要动态设置优化器类型的上下文中，例如，它可能被传递给一个函数或方法，该函数或方法会根据这个字符串来设置其内部的优化器类型。
'''
# Learning rate schedule
lr_scheduler_type = "cosine"
'''
这行代码定义了一个变量 `lr_scheduler_type`，并将其值设置为字符串 `"cosine"`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`lr_scheduler_type` 可能是用来指定学习率调度器的类型。在深度学习中，学习率调度器是用于在训练过程中动态调整学习率的策略。常见的学习率调度器包括固定学习率、时间衰减、步长衰减等。

字符串 `"cosine"` 可能代表一种特定的学习率调度器，即余弦退火调度器（Cosine Annealing Scheduler）。余弦退火调度器会根据余弦函数的形状来调整学习率，使得学习率在训练开始时较大，然后逐渐减小，最后接近零。这种调度器可以在训练开始时快速探索参数空间，然后在训练结束时更精细地优化参数。

请注意，这里 `"cosine"` 是一个字符串，而不是直接的调度器类型。这意味着它可能被用在一个需要动态设置调度器类型的上下文中，例如，它可能被传递给一个函数或方法，该函数或方法会根据这个字符串来设置其内部的调度器类型。
'''
# Number of training steps (overrides num_train_epochs)
max_steps = -1
'''
这行代码定义了一个变量 `max_steps`，并将其值设置为整数 `-1`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`max_steps` 是用来指定训练步数的。在深度学习中，一步训练通常指的是对一个批量的数据进行一次前向和反向传播。例如，如果你有 1000 个训练样本，批量大小为 4，那么一个 epoch 就包含 250 步训练。

设置 `max_steps = -1` 表示不限制训练步数。这可能意味着训练将持续进行，直到满足其他停止条件，例如达到预设的 epoch 数量或满足早停条件。然而，注释中提到了 "overrides num_train_epochs"，这可能意味着如果 `max_steps` 的值大于 0，那么它将覆盖 `num_train_epochs`，训练将在达到 `max_steps` 步后停止，而不管是否已经完成了预设的 epoch 数量。

请注意，这里的 `-1` 是一个整数，而不是字符串。在 Python 中，整数可以直接用于数学运算，而不需要进行任何转换。
'''
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
'''
这行代码定义了一个变量 `warmup_ratio`，并将其值设置为浮点数 `0.03`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`warmup_ratio` 是用来指定线性预热（linear warmup）的步数比例。在深度学习中，预热是一种常用的学习率调度策略，它在训练开始时逐渐增加学习率，直到达到预设的学习率。这种策略可以在训练开始时避免参数更新过大，从而提高模型的稳定性和性能。

设置 `warmup_ratio = 0.03` 表示预热步数为总训练步数的 3%。例如，如果总训练步数为 10000 步，那么在前 300 步（即 10000 * 0.03）中，学习率将从 0 线性增加到预设的学习率。

请注意，这里的 `0.03` 是一个浮点数，而不是字符串。在 Python 中，浮点数可以直接用于数学运算，而不需要进行任何转换。
'''
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
'''
这行代码定义了一个变量 `group_by_length`，并将其值设置为布尔值 `True`。在 Python 中，`=` 用于赋值，即将等号右侧的值赋给等号左侧的变量。

在这个上下文中，`group_by_length` 可能是用来控制是否将相同长度的序列分组到同一个批量中的开关。在深度学习中，特别是在处理序列数据（如文本或时间序列）时，将相同长度的序列分组到同一个批量中可以节省内存并加速训练。这是因为在处理不同长度的序列时，通常需要对较短的序列进行填充（padding）以匹配最长的序列，而将相同长度的序列分组可以减少这种填充。

设置 `group_by_length = True` 表示启用这种分组策略。这可以在处理大量不同长度的序列时，提高训练的效率和性能。

请注意，这里的 `True` 是一个布尔值，而不是字符串。在 Python 中，布尔值只有两种：`True` 和 `False`，分别代表逻辑真和逻辑假。
'''
# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.350100
2,2.015800
3,1.048700
4,1.287700
5,1.451200
6,1.659900
7,1.472300
8,1.326700
9,1.140000
10,1.395300


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<s>[INST] What is a large language model? [/INST] A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate human-like language outputs. It is designed to be able to understand and generate text in a way that is similar to human language, and can be used for a wide range of applications such as chatbots, language translation, and text summarization.

Large language models are typically trained using deep learning techniques, such as recurrent neural networks (RNNs) or transformer models, and are often based on pre-trained models such as BERT or RoBERTa. These models are trained on large datasets of text, such as books, articles, or websites, and are designed to learn the patterns and structures of language.

Some examples of large language models include:

* BERT (Bidirectional Encoder Representations from Transformers


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

19965

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mlabonne/llama-2-7b-miniguanaco/commit/c81a32fd0b4d39e252326e639d63e75aa68c9a4a', commit_message='Upload tokenizer', commit_description='', oid='c81a32fd0b4d39e252326e639d63e75aa68c9a4a', pr_url=None, pr_revision=None, pr_num=None)